## 함수 감싸기 

In [2]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

if __name__ == '__main__':
    @timethis
    def countdown(n:int):
        while n > 0:
            n -= 1

    countdown(100000)
    countdown(10000000)

countdown 0.00745391845703125
countdown 0.5320258140563965


## 함수 메타데이터 보존 

In [3]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

if __name__ == '__main__':
    @timethis
    def countdown(n:int):
        '''
        Counts down
        '''
        while n > 0:
            n -= 1

    countdown(100000)
    print('Name:', countdown.__name__)
    print('Docstring:', repr(countdown.__doc__))
    print('Annotations:', countdown.__annotations__)

countdown 0.007941484451293945
Name: countdown
Docstring: '\n        Counts down\n        '
Annotations: {'n': <class 'int'>}


## 데코레이터 풀기 

In [4]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def add(x, y):
    return x + y

print(add(2,3))

print(add.__wrapped__(2,3))

Decorator 1
Decorator 2
5
Decorator 2
5


## 매개변수를 받는 데코레이터 정의

In [5]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    print(add(2,3))
    spam()

DEBUG:__main__:add
CRITICAL:example:spam


5
Spam!


## 사용자가 조절 가능한 속성을 가진 데코레이터

In [13]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    print(f'obj {obj}, func.__name__:{func.__name__} func {func}')
    # wrapper를 통해서 반환된 add 에 attribute로 set_level, set_message를 추가해주는 부분 
    return func

def logged(level, name=None, message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper
    return decorate

@logged(logging.DEBUG)
def add(x, y):
    return x + y
# add = logged(logging.DEBUG)(add)

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

import time
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return r
    return wrapper

@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1

@logged(logging.DEBUG)
@timethis
def countdown2(n):
    while n > 0:
        n -= 1

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)

    print(f'add address {add}')
    print(add(2, 3))

    add.set_message('Add called')
    print(add(2, 3))

    add.set_level(logging.WARNING)
    print(add(2, 3))

    countdown(100000)
    countdown.set_level(logging.CRITICAL)
    countdown(100000)

    countdown2(100000)
    countdown2.set_level(logging.CRITICAL)
    countdown2(100000)


DEBUG:__main__:add
DEBUG:__main__:Add called
DEBUG:__main__:countdown
CRITICAL:__main__:countdown
DEBUG:__main__:countdown2
CRITICAL:__main__:countdown2


obj <function add at 0x7f751caf3f28>, func.__name__:set_level func <function logged.<locals>.decorate.<locals>.set_level at 0x7f751caf3ea0>
obj <function add at 0x7f751caf3f28>, func.__name__:set_message func <function logged.<locals>.decorate.<locals>.set_message at 0x7f751caf3e18>
obj <function spam at 0x7f751caf3c80>, func.__name__:set_level func <function logged.<locals>.decorate.<locals>.set_level at 0x7f751caf3bf8>
obj <function spam at 0x7f751caf3c80>, func.__name__:set_message func <function logged.<locals>.decorate.<locals>.set_message at 0x7f751caf3b70>
obj <function countdown at 0x7f751caf3ae8>, func.__name__:set_level func <function logged.<locals>.decorate.<locals>.set_level at 0x7f751caf3a60>
obj <function countdown at 0x7f751caf3ae8>, func.__name__:set_message func <function logged.<locals>.decorate.<locals>.set_message at 0x7f751caf39d8>
obj <function countdown2 at 0x7f751caf3048>, func.__name__:set_level func <function logged.<locals>.decorate.<locals>.set_level at 0x7

## 옵션 매개변수를 받는 데코레이터

In [14]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

@logged
def add(x, y):
    return x + y

@logged()
def sub(x, y):
    return x - y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    add(2,3)
    sub(2,3)
    spam()


DEBUG:__main__:add
DEBUG:__main__:sub
CRITICAL:example:spam


Spam!
